# Project testing

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=RapekuOl9OxKxJpNRddcmYlJ-Te8Ay9-zMUfGgQJpZo&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/uwEOFqDUUMKjRc11zFYWnb9FaRO1-3GoF3ybzEaIWfIa8OvIp5HbcxA
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [0]:
%env GCLOUD_PROJECT=reddit-master

env: GCLOUD_PROJECT=reddit-master


In [0]:
import numpy as np 
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import re
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
import plotly.figure_factory as ff
import logging
import nltk
nltk.download('stopwords')
import keras.backend as K

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import log_loss
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from tensorflow import metrics, local_variables_initializer
from keras.models import load_model


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


### Downloading the model

In [0]:
K.clear_session()

def auc(y_true, y_pred):
    auc = metrics.auc(y_true, y_pred)[1]
    K.get_session().run(local_variables_initializer())
    return auc

In [0]:
dependencies = {
    'auc': auc
}

In [0]:
!gsutil cp gs://reddit_models/auc_model_lstm_30_batchsize_150_10_subreddits.h5 .

Copying gs://reddit_models/auc_model_lstm_30_batchsize_150_10_subreddits.h5...
| [1 files][ 86.0 MiB/ 86.0 MiB]                                                
Operation completed over 1 objects/86.0 MiB.                                     


In [0]:
model = load_model("auc_model_lstm_30_batchsize_150_10_subreddits.h5", custom_objects=dependencies)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
# importing my final tokenizer

!gsutil cp gs://reddit_models/reddit_tokenizer.pkl .

Copying gs://reddit_models/reddit_tokenizer.pkl...
\ [1 files][ 47.0 MiB/ 47.0 MiB]                                                
Operation completed over 1 objects/47.0 MiB.                                     


In [0]:
with open('reddit_tokenizer.pkl', 'rb') as file:
    tokenizer = pkl.load(file)

In [0]:
def label_decoder(subreddit):
    dict_labels = {
        0:"Fitness",
        1:"atheism",
        2:"aww",
        3:"europe",
        4:"gaming",
        5:"movies",
        6:"nba",
        7:"politics",
        8:"science",
        9:"technology"
    }
    
    return dict_labels[subreddit]

# How test my model

Here below you can find the steps to follow for testing the model:

    

*   go to reddit.com and search for one of the following subreddits:

        Fitness
        atheism
        aww
        europe
        gaming
        movies
        nba
        politics
        science
        technology

*   copy the text you want to test

*   go to "Step 1" and paste the text once you have executed the correspoding row

*   Execute step 2

*   Execute step 3 for the result

**Final test**: is the suggestion of my model exactly the subreddit where you picked the text?!

## Step 1: copy the text or post you what to publush

In [0]:
text = input("Please, enter the text of your blog: ") 
print("")
print("Text correctly entered, I'll give you that subreddit in a minute")

Please, enter the text of your blog: As you can see in the picture below, I made it to quite a few places over my time abroad. I started things off in Iceland for a short stay before I moved to Germany, followed by the Czech Republic and Austria. From there, I went down to Greece and Italy for nearly 3 weeks. I moved up back into Germany again and over to Hungary and Poland before I caught a flight to London. I went to Amsterdam, Brussels, and Paris for a bit and eventually made my way down to Spain where I ended up staying for over two weeks. Wrapping things up, I went over to Portugal and then spent some time in Ireland.

Text correctly entered, I'll give you that subreddit in a minute


## Step 2: processing the text

In [0]:
MAX_NB_WORDS = 75000
MAX_SEQUENCE_LENGTH = 450
EMBEDDING_DIM = 100

X = tokenizer.texts_to_sequences([text])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (1, 450)


## Step 3: get the result

In [0]:
result = model.predict(X)

def convert_to_cat(arr):
    biggest = 0
    for x in range(0, len(arr)):
        if arr[x] > arr[biggest]:
            biggest = x
    return biggest

category = [label_decoder(convert_to_cat(x)) for x in result]

print("you should publish your text into the sureddit: ", category)

you should publish your text into the sureddit:  ['europe']
